In [1]:
from simtk import unit, openmm
from simtk.openmm import app
import numpy as np
from openeye import oechem
import os, smarty, parmed, openmoltools, pdbfixer 
from openmoltools import forcefield_generators

In [2]:
# Generate the SMIRNOFF ligand Structure object
ifs = oechem.oemolistream('../OpenMMCubes/tests/input/toluene.pdb')
mol = oechem.OEMol()
flavor = oechem.OEIFlavor_Generic_Default | oechem.OEIFlavor_MOL2_Default | oechem.OEIFlavor_MOL2_Forcefield
ifs.SetFlavor( oechem.OEFormat_MOL2, flavor)
oechem.OEReadMolecule(ifs, mol )
oechem.OETriposAtomNames(mol)

from smarty.forcefield import ForceField
mol_ff = ForceField('../OpenMMCubes/tests/input/forcefield/smirff99Frosst.ffxml')
mol_top, mol_sys, mol_pos = smarty.forcefield_utils.create_system_from_molecule(mol_ff, mol)
molecule_structure = parmed.openmm.load_topology(mol_top, mol_sys) 
molecule_structure.residues[0].name = "MOL"

print(molecule_structure)
print(molecule_structure.topology)

<Structure 15 atoms; 1 residues; 15 bonds; parametrized>
<Topology; 1 chains, 1 residues, 15 atoms, 15 bonds>


In [3]:
# Generate protein Stucture object
pdbfilename = os.path.join('..','OpenMMCubes', 'tests', 'input', 'T4-protein.pdb')
pdbfile = app.PDBFile(pdbfilename)
forcefield = app.ForceField('amber99sbildn.xml', 'tip3p.xml')
system = forcefield.createSystem( pdbfile.topology )
protein_structure = parmed.openmm.load_topology( pdbfile.topology, system)

print(protein_structure)
print(protein_structure.topology)

<Structure 2634 atoms; 164 residues; 2654 bonds; parametrized>
<Topology; 1 chains, 164 residues, 2634 atoms, 2654 bonds>


In [4]:
# Merge structures
structure = protein_structure + molecule_structure
topology = structure.topology

# Concatenate positions arrays
positions_unit = unit.angstroms
positions0_dimensionless = np.array( pdbfile.positions / positions_unit )
positions1_dimensionless = np.array( mol_pos / positions_unit )

coordinates = np.vstack((positions0_dimensionless,positions1_dimensionless))
natoms = len(coordinates)
positions = np.zeros([natoms,3], np.float32)
for index in range(natoms):
    (x,y,z) = coordinates[index]
    positions[index,0] = x
    positions[index,1] = y
    positions[index,2] = z
positions = unit.Quantity(positions, positions_unit)

# Store in Structure object
structure.coordinates = coordinates

# Save to PDB
structure.save('tmp.pdb',overwrite=True)

In [5]:
# Solvate with PDBFixer
fixer = pdbfixer.PDBFixer('tmp.pdb')
#fixer.findMissingResidues()
#fixer.findMissingAtoms()
#fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
fixer.addSolvent(padding=10*unit.angstrom, ionicStrength=0.050*unit.molar)

In [6]:
# Load PDBFixer object back to Structure
struct = parmed.openmm.load_topology(fixer.topology, xyz=fixer.positions)
# Remove from ligand from protein Structure
struct.strip(":MOL")
struct

<Structure 32003 atoms; 9971 residues; 22216 bonds; PBC (orthogonal); NOT parametrized>

In [7]:
# Regenerate OpenMM System to parameterize solvent
system = forcefield.createSystem(struct.topology, rigidWater=False)

# Regenerate parameterized protein structure
protein_structure = parmed.openmm.load_topology( struct.topology, system=system, 
                                                xyz=struct.positions, box=struct.box )

In [8]:
# Remerge with ligand structure
combined_structure = protein_structure + molecule_structure

# Restore initial positions and box dimensions
combined_structure.positions = fixer.positions
combined_structure.box = struct.box
os.remove('tmp.pdb')
combined_structure.save('combined_structure.pdb', overwrite=True)

In [9]:
# Regenerate OpenMM system with ParmEd
system = combined_structure.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=10.0*unit.angstroms, 
                                constraints=app.HBonds)

In [10]:
pdbfilename = 'combined_structure.pdb'
pdbfile = app.PDBFile(pdbfilename)
topology = pdbfile.getTopology()
positions = pdbfile.getPositions()

In [11]:
integrator = openmm.LangevinIntegrator(310*unit.kelvin, 1/unit.picoseconds, 0.002*unit.picoseconds)
simulation = app.Simulation(topology, system, integrator )

In [12]:
simulation.context.setPositions(positions)
simulation.context.setVelocitiesToTemperature(310*unit.kelvin)
simulation.minimizeEnergy()

In [26]:
statereporter = app.StateDataReporter('output.log', 1000, step=True, potentialEnergy=True, temperature=True)
simulation.reporters.append(statereporter)

In [20]:
print(statereporter)
dir(statereporter)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_checkForErrors',
 '_constructHeaders',
 '_constructReportValues',
 '_density',
 '_elapsedTime',
 '_hasInitialized',
 '_initializeConstants',
 '_kineticEnergy',
 '_needEnergy',
 '_needsForces',
 '_needsPositions',
 '_needsVelocities',
 '_openedFile',
 '_out',
 '_potentialEnergy',
 '_progress',
 '_remainingTime',
 '_reportInterval',
 '_separator',
 '_speed',
 '_step',
 '_temperature',
 '_time',
 '_totalEnergy',
 '_totalMass',
 '_totalSteps',
 '_volume',
 'describeNextReport',
 'report']

In [27]:
simulation.step(10000)

In [28]:
state = simulation.context.getState( getPositions=True,
                                     getVelocities=True,
                                     getForces=True,
                                     getEnergy=True,
                                     getParameters=True,
                                     enforcePeriodicBox=True )

In [32]:
print(statereporter)